# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split

# Read the CSV into DataFrame

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Load the data
file_path = Path('/content/drive/MyDrive/11/Resources/lending_data.csv')
df = pd.read_csv(file_path)
display(df.head())
print(df.shape)

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


(77536, 9)


In [5]:
lis1=[]
for i in df.columns:
  if(df[i].dtype=='object' and i!='loan_status'):
    lis1.append(i)

print(lis1)

['homeowner']


# Split the Data into Training and Testing

In [6]:
# Create our features
X = df.drop(columns="loan_status") # YOUR CODE HERE
X = pd.get_dummies(X,columns=lis1)

# Create our target
y = df['loan_status'].to_frame()

In [7]:
X.describe().T

,count,mean,std,min,25%,50%,75%,max
loan_size,77536.0,9805.562577,2093.223153,5000.00,8700.000000,9500.000000,10400.000000,23800.000000
interest_rate,77536.0,7.292333,0.889495,5.25,6.825000,7.172000,7.528000,13.235000
borrower_income,77536.0,49221.949804,8371.635077,30000.00,44800.000000,48100.000000,51400.000000,105200.000000
debt_to_income,77536.0,0.377318,0.081519,0.00,0.330357,0.376299,0.416342,0.714829
num_of_accounts,77536.0,3.826610,1.904426,0.00,3.000000,4.000000,4.000000,16.000000
derogatory_marks,77536.0,0.392308,0.582086,0.00,0.000000,0.000000,1.000000,3.000000
total_debt,77536.0,19221.949804,8371.635077,0.00,14800.000000,18100.000000,21400.000000,75200.000000
homeowner_mortgage,77536.0,0.497472,0.499997,0.00,0.000000,0.000000,1.000000,1.000000
homeowner_own,77536.0,0.398911,0.489678,0.00,0.000000,0.000000,1.000000,1.000000
homeowner_rent,77536.0,0.103616,0.304764,0.00,0.000000,0.000000,0.000000,1.000000


In [8]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [9]:
# Create X_train, X_test, y_train, y_test
# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1,test_size=0.33)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [10]:
from pandas.core.common import random_state
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
# YOUR CODE HERE
scalar = StandardScaler()

In [11]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE
scalar.fit(X_train)

StandardScaler()

In [12]:
# Scale the training and testing data
# YOUR CODE HERE
scaled_X_train = scalar.transform(X_train)
scaled_X_test = scalar.transform(X_test)

# Simple Logistic Regression

In [13]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [14]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.948898931463884

In [15]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  761,    82],
       [  122, 24622]])

In [16]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.90      1.00      0.88      0.95      0.89       843
   low_risk       1.00      1.00      0.90      1.00      0.95      0.91     24744

avg / total       0.99      0.99      0.91      0.99      0.95      0.91     25587



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [17]:
from imblearn.over_sampling import RandomOverSampler
# Resample the training data with the RandomOversampler
# YOUR CODE HERE
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
# View the count of target classes with Counter
# YOUR CODE HERE
print(Counter(y_resampled))
print(y_resampled.value_counts())


Counter({'loan_status': 1})
loan_status
high_risk      50292
low_risk       50292
dtype: int64


In [18]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model = LogisticRegression(random_state=1)
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test)

In [19]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
print(balanced_accuracy_score(y_test, y_pred))

0.9947985521203314


In [20]:
# Display the confusion matrix
# YOUR CODE HERE
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  839,     4],
       [  140, 24604]])

In [21]:
# Print the imbalanced classification report
# YOUR CODE HERE
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      1.00      0.99      0.92      0.99      0.99       843
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     24744

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     25587



### SMOTE Oversampling

In [22]:
# Resample the training data with SMOTE
# YOUR CODE HERE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1).fit_resample(X_train, y_train)
# View the count of target classes with Counter
# YOUR CODE HERE
print(Counter(y_resampled))
print(y_resampled.value_counts())

Counter({'loan_status': 1})
loan_status
high_risk      50292
low_risk       50292
dtype: int64


In [23]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model = LogisticRegression(random_state=1)
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test)

In [24]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
balanced_accuracy_score(y_test, y_pred)

0.9947985521203314

In [25]:
# Display the confusion matrix
# YOUR CODE HERE
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  839,     4],
       [  140, 24604]])

In [26]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      1.00      0.99      0.92      0.99      0.99       843
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     24744

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     25587



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [27]:
# Resample the data using the ClusterCentroids resampler
# YOUR CODE HERE
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)


# View the count of target classes with Counter
# YOUR CODE HERE
print(Counter(y_resampled))
print(y_resampled.value_counts())


Counter({'loan_status': 1})
loan_status
high_risk      1657
low_risk       1657
dtype: int64


In [28]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model = LogisticRegression(random_state=1)
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test)

In [29]:
# Calculate the balanced accuracy score
# YOUR CODE HERE
print(balanced_accuracy_score(y_test, y_pred))

0.9860027895615515


In [30]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_pred)

array([[  824,    19],
       [  135, 24609]])

In [31]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.98      0.99      0.91      0.99      0.97       843
   low_risk       1.00      0.99      0.98      1.00      0.99      0.97     24744

avg / total       0.99      0.99      0.98      0.99      0.99      0.97     25587



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [32]:
from imblearn.combine import SMOTEENN
# Resample the training data with SMOTEENN
# YOUR CODE HERE
sm = SMOTEENN(random_state=1)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)

# View the count of target classes with Counter
# YOUR CODE HERE
print(Counter(y_resampled))
print(y_resampled.value_counts())


Counter({'loan_status': 1})
loan_status
low_risk       49975
high_risk      49570
dtype: int64


In [33]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model = LogisticRegression(random_state=1)
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test)

In [34]:
# Calculate the balanced accuracy score
# YOUR CODE HERE
balanced_accuracy_score(y_test, y_pred)

0.9945964829318412

In [35]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_pred)

array([[  839,     4],
       [  150, 24594]])

In [36]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       843
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     24744

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     25587



# Final Questions

1. Which model had the best balanced accuracy score?

   Combination SMOTE has best accuracy score

2. Which model had the best recall score?

   Recall score for all the models is same

3. Which model had the best geometric mean score?

   Geometric mean score for all the models is same 
